### 직방
- 동 이름을 입력하면 아파트 매물을 데이터 프레임으로 만들기
- robots.txt 

1.  웹서비스 분석: url찾기, 크롤링 전략 세우기
 - 동이름 입력 -> lat, lan
 - lat, lag -> geohash: geohash 패키지 설치 
 - geohasg -> item ids
 - item ids -> item datas(데이터 프레임)

In [20]:
# 동이름 -> 위도 경도 
# google에서 uriencode 사이트 찾아서 url 입력 -> 의미 찾기
addr = "성수동"
url = "https://apis.zigbang.com/v2/search?q={}".format(addr)
response = requests.get(url)
# 성수동 1가 데이터만 추리기
data =  response.json()["items"][0]
# 위도, 경도 데이터 가져오기
lat, lng = data["lat"], data["lng"]
lat, lng

data


{'success': True,
 'code': '200',
 'items': [{'id': 4094,
   'type': 'address',
   'name': '성수동1가',
   'hint': '',
   'description': '서울시 성동구 성수동1가',
   'lat': 37.541725158691406,
   'lng': 127.04335021972656,
   'zoom': 5,
   'polygon': [],
   '_score': None,
   '_source': {'name_length': 5,
    'local1': '서울시',
    'local2': '성동구',
    'local3': '성수동1가',
    'web_level': 15,
    'web_lat': 37.541725158691406,
    'web_lng': 127.04335021972656,
    'app_level': 15,
    'app_lat': 37.541725158691406,
    'app_lng': 127.04335021972656,
    '법정동코드': '1120011400'},
   'zoom_level': {'google': 15, 'daum': 4},
   'zoom_level_v2': {'app': 5, 'web': 4}},
  {'id': 4095,
   'type': 'address',
   'name': '성수동2가',
   'hint': '',
   'description': '서울시 성동구 성수동2가',
   'lat': 37.54096221923828,
   'lng': 127.05702209472656,
   'zoom': 5,
   'polygon': [],
   '_score': None,
   '_source': {'name_length': 5,
    'local1': '서울시',
    'local2': '성동구',
    'local3': '성수동2가',
    'web_level': 15,
    'web

In [2]:
# 위도, 경도 -> geohash
# pip install geohash2

In [3]:
pip install geohash2

Note: you may need to restart the kernel to use updated packages.


In [4]:
import geohash2

In [5]:
# precision : 작을수록 큰범위, 클수록 작은 범위
geohash = geohash2.encode(lat, lng, precision = 5)
lat, lng, geohash

(37.541725158691406, 127.04335021972656, 'wydmd')

In [30]:
# geohash -> ids
url = "https://apis.zigbang.com/property/apartments/items?\
domain=zigbang&geohash={}\
&q=type=sales%7Cprice=0~-1%7CfloorArea=0~-1".format(geohash)
response = requests.get(url)
datas = response.json()['items']
ids = [data["itemId"] for data in datas]
len(ids), ids[:3]

(844, [23556619, 24302097, 23639002])

In [31]:
datas[0]

{'itemId': 23556619,
 'lat': 37.55790135050871,
 'lng': 127.02711722579186,
 'created_at': '2020-08-21 12:54:34'}

In [32]:
# ids -> items
ids_str = str(ids[:200]).replace(" ", "")
url = "https://apis.zigbang.com/property/apartments/items?\
vritemIds={ids}&itemIds={ids}&citemIds={ids}".format(ids=ids_str)
response = requests.get(url)
datas = response.json()["items"]
datas

[{'itemId': 23556619,
  'type': '매매',
  'exclusive_type': '1',
  'rent': 0,
  'buildingFloor': '15',
  'itemImages': None,
  'house_key': None,
  'lat': 37.55790135050871,
  'lng': 127.02711722579186,
  'vr_status': None,
  'vr_request_id': None,
  'itemTitle': '추천 정남향 씽크대 욕실부분수리 아주깨끗 현재 신혼거주 ',
  'agentNo': None,
  'regDate': '2020-10-27',
  'regTime': '2020-10-27T04:47:50.000Z',
  'apartmentId': 5361,
  'apartmentName': '행당대림',
  'buildingName': '107동',
  'roomTypeId': 8646,
  'grossAreaName': {'m2': 83, 'p': 25},
  'grossArea': {'m2': 83, 'p': 25},
  'agreementArea': {'m2': 0, 'p': 0},
  'netArea': {'m2': 59, 'p': 17},
  'floorPlan': 'https://ic.zigbang.com/ic/floorplan/5361/8646/02844f3d87f04b03aeb12b6f76e051b8.png',
  'userNo': 6101798,
  'agentId': 6101798,
  'agentName': '굿모닝공인중개사사무소',
  'agentTel': '0222912227',
  'agentUserName': '신진옥',
  'agentPhone': '01052639454',
  'created_at': '2020-08-21 12:54:34',
  'real_type': '아파트',
  'itemImage': ['https://ic.zigbang.com/ic/floorpl

In [33]:
item_df = pd.DataFrame(datas)
item_df.tail()

,itemId,type,exclusive_type,rent,buildingFloor,itemImages,house_key,lat,lng,vr_status,...,agentPhone,created_at,real_type,itemImage,sectionType,viewType,groupedItemFloor,agreementAreaName,isNew,sales
195,24369753,매매,1,0,18,None,None,37.558615,127.024705,None,...,01043873588,2020-10-16 15:12:53,아파트,[https://ic.zigbang.com/ic/floorplan/6323/1031...,premium_recommend,1,저층,"{'m2': 74, 'p': 22}",True,90000
196,22928671,매매,1,0,15,None,None,37.556604,127.031756,None,...,01043873588,2020-07-13 13:03:12,아파트,[https://ic.zigbang.com/ic/floorplan/18421/152...,premium_recommend,1,고층,"{'m2': 82, 'p': 24}",True,120000
197,23042237,매매,1,0,15,None,None,37.556604,127.031756,None,...,01043873588,2020-07-20 12:57:52,아파트,[https://ic.zigbang.com/ic/floorplan/18421/152...,premium_recommend,1,고층,"{'m2': 82, 'p': 24}",True,120000
198,23172133,매매,1,0,14,None,None,37.556604,127.031756,None,...,01043873588,2020-07-28 12:45:06,아파트,[https://ic.zigbang.com/ic/floorplan/18421/152...,premium_recommend,1,고층,"{'m2': 82, 'p': 24}",True,120000
199,23599698,매매,1,0,15,None,None,37.556604,127.031756,None,...,01043873588,2020-08-24 12:03:16,아파트,[https://ic.zigbang.com/ic/floorplan/18421/152...,premium_recommend,1,고층,"{'m2': 143, 'p': 43}",True,183000


In [34]:
len(item_df.columns)

39

In [36]:
pd.options.display.max_columns = len(item_df.columns)

In [37]:
item_df.tail()

,itemId,type,exclusive_type,rent,buildingFloor,itemImages,house_key,lat,lng,vr_status,vr_request_id,itemTitle,agentNo,regDate,regTime,apartmentId,apartmentName,buildingName,roomTypeId,grossAreaName,grossArea,agreementArea,netArea,floorPlan,userNo,agentId,agentName,agentTel,agentUserName,agentPhone,created_at,real_type,itemImage,sectionType,viewType,groupedItemFloor,agreementAreaName,isNew,sales
195,24369753,매매,1,0,18,None,None,37.558615,127.024705,None,None,올확장하고 화장실수리된집.,NaN,2020-10-26,2020-10-26T08:25:00.000Z,6323,왕십리풍림아이원,110동,10317,"{'m2': 74, 'p': 22}","{'m2': 74, 'p': 22}","{'m2': 0, 'p': 0}","{'m2': 59, 'p': 17}",https://ic.zigbang.com/ic/floorplan/6323/10317...,2724034,2724034,우리공인중개사,0222991900,최성숙,01043873588,2020-10-16 15:12:53,아파트,[https://ic.zigbang.com/ic/floorplan/6323/1031...,premium_recommend,1,저층,"{'m2': 74, 'p': 22}",True,90000
196,22928671,매매,1,0,15,None,None,37.556604,127.031756,None,None,남향 올확장 깨끗한집 행당역초역세권,NaN,2020-10-26,2020-10-26T08:25:00.000Z,18421,두산위브,101동,15203,"{'m2': 82, 'p': 24}","{'m2': 82, 'p': 24}","{'m2': 0, 'p': 0}","{'m2': 59, 'p': 17}",https://ic.zigbang.com/ic/floorplan/18421/1520...,2724034,2724034,우리공인중개사,0222991900,최성숙,01043873588,2020-07-13 13:03:12,아파트,[https://ic.zigbang.com/ic/floorplan/18421/152...,premium_recommend,1,고층,"{'m2': 82, 'p': 24}",True,120000
197,23042237,매매,1,0,15,None,None,37.556604,127.031756,None,None,2년전 올수리 주인거주로 관리상태좋음,NaN,2020-10-26,2020-10-26T08:25:00.000Z,18421,두산위브,101동,15203,"{'m2': 82, 'p': 24}","{'m2': 82, 'p': 24}","{'m2': 0, 'p': 0}","{'m2': 59, 'p': 17}",https://ic.zigbang.com/ic/floorplan/18421/1520...,2724034,2724034,우리공인중개사,0222991900,최성숙,01043873588,2020-07-20 12:57:52,아파트,[https://ic.zigbang.com/ic/floorplan/18421/152...,premium_recommend,1,고층,"{'m2': 82, 'p': 24}",True,120000
198,23172133,매매,1,0,14,None,None,37.556604,127.031756,None,None,올확장한집 초역세권 아파트 (세안고),NaN,2020-10-26,2020-10-26T08:25:00.000Z,18421,두산위브,102동,15203,"{'m2': 82, 'p': 24}","{'m2': 82, 'p': 24}","{'m2': 0, 'p': 0}","{'m2': 59, 'p': 17}",https://ic.zigbang.com/ic/floorplan/18421/1520...,2724034,2724034,우리공인중개사,0222991900,최성숙,01043873588,2020-07-28 12:45:06,아파트,[https://ic.zigbang.com/ic/floorplan/18421/152...,premium_recommend,1,고층,"{'m2': 82, 'p': 24}",True,120000
199,23599698,매매,1,0,15,None,None,37.556604,127.031756,None,None,올확장된집. 깨끗하고 조용한단지.초역세권 아파트,NaN,2020-10-26,2020-10-26T08:25:00.000Z,18421,두산위브,103동,15202,"{'m2': 143, 'p': 43}","{'m2': 143, 'p': 43}","{'m2': 0, 'p': 0}","{'m2': 114, 'p': 34}",https://ic.zigbang.com/ic/floorplan/18421/1520...,2724034,2724034,우리공인중개사,0222991900,최성숙,01043873588,2020-08-24 12:03:16,아파트,[https://ic.zigbang.com/ic/floorplan/18421/152...,premium_recommend,1,고층,"{'m2': 143, 'p': 43}",True,183000


In [39]:
columns = [ "itemId", "buildingFloor", "groupedItemFloor", "grossArea", "lat", "lng", "sales", "itemTitle" ]
item_df = item_df[columns]
item_df.tail(2)

,itemId,buildingFloor,groupedItemFloor,grossArea,lat,lng,sales,itemTitle
198,23172133,14,고층,"{'m2': 82, 'p': 24}",37.556604,127.031756,120000,올확장한집 초역세권 아파트 (세안고)
199,23599698,15,고층,"{'m2': 143, 'p': 43}",37.556604,127.031756,183000,올확장된집. 깨끗하고 조용한단지.초역세권 아파트


In [41]:
item_df["m2"] = item_df["grossArea"].apply(lambda data: data["m2"])
item_df["p"] = item_df["grossArea"].apply(lambda data: data["p"])
item_df.drop(columns=["grossArea"], inplace=True)

In [42]:
item_df.tail(1)

,itemId,buildingFloor,groupedItemFloor,lat,lng,sales,itemTitle,m2,p
199,23599698,15,고층,37.556604,127.031756,183000,올확장된집. 깨끗하고 조용한단지.초역세권 아파트,143,43


### 함수로 만들기 
- 동이름 -> 아파트 매물 데이터 프레임

In [43]:
def crawling_apt(addr):
    # 동이름 -> 위도,경도
    url = "https://apis.zigbang.com/search?q={}".format(addr)
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    # 위도,경도 -> geohash
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # geohash -> ids
    url = "https://apis.zigbang.com/property/apartments/items?\
domain=zigbang&geohash={}\
&q=type=sales%7Cprice=0~-1%7CfloorArea=0~-1".format(geohash)
    response = requests.get(url)
    datas = response.json()["items"]
    ids = [data["itemId"] for data in datas]
    
    # ids -> items : 200개씩
    dfs = []
    for idx in range(0, len(ids), 200):
        start, end =  idx, idx + 200
        ids_str = str(ids[start:end]).replace(" ", "")
        url = "https://apis.zigbang.com/property/apartments/items?\
        vritemIds={ids}&itemIds={ids}&citemIds={ids}".format(ids=ids_str)
        response = requests.get(url)
        datas = response.json()["items"]
        item_df = pd.DataFrame(datas)
        columns = [ "itemId", "buildingFloor", "groupedItemFloor", "grossArea", "lat", "lng", "sales", "itemTitle" ]
        item_df = item_df[columns]
        item_df["m2"] = item_df["grossArea"].apply(lambda data: data["m2"])
        item_df["p"] = item_df["grossArea"].apply(lambda data: data["p"])
        item_df.drop(columns=["grossArea"], inplace=True)
        dfs.append(item_df)
        
    result_df = pd.concat(dfs)
    result_df.reset_index(drop = True, inplace = True)
    return result_df

In [1]:
%%writefile zigbang.py
import pandas as pd
import requests
import geohash2

def crawling_apt(addr):
    # 동이름 -> 위도,경도
    url = "https://apis.zigbang.com/search?q={}".format(addr)
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    # 위도,경도 -> geohash
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # geohash -> ids
    url = "https://apis.zigbang.com/property/apartments/items?\
domain=zigbang&geohash={}\
&q=type=sales%7Cprice=0~-1%7CfloorArea=0~-1".format(geohash)
    response = requests.get(url)
    datas = response.json()["items"]
    ids = [data["itemId"] for data in datas]
    
    # ids -> items : 200개씩
    dfs = [] 
    for idx in range(0, len(ids), 200):
        start, end = idx, idx + 200
        ids_str = str(ids[start:end]).replace(" ", "")
        url = "https://apis.zigbang.com/property/apartments/items?\
        vritemIds={ids}&itemIds={ids}&citemIds={ids}".format(ids=ids_str)
        response = requests.get(url)
        datas = response.json()["items"]
        item_df = pd.DataFrame(datas)
        columns = [ "itemId", "buildingFloor", "groupedItemFloor", "grossArea", "lat", "lng", "sales", "itemTitle" ]
        item_df = item_df[columns]
        item_df["m2"] = item_df["grossArea"].apply(lambda data: data["m2"])
        item_df["p"] = item_df["grossArea"].apply(lambda data: data["p"])
        item_df.drop(columns=["grossArea"], inplace=True)
        dfs.append(item_df)
    result_df = pd.concat(dfs)
    result_df.reset_index(drop=True, inplace=True)
    return result_df

Writing zigbang.py
